In [1]:
import numpy as np
import pyccl as ccl
import cluster_toolkit as ct

In [19]:
# Define CCL cosmology object
cosmo = ccl.Cosmology(Omega_c=0.27, Omega_b=0.045, h=0.67, A_s=2.1e-9, n_s=0.96)

# Select density profile and profile parametrization options 
density_profile_parametrization = 'nfw'
mass_Delta = 200
cluster_mass = 1.e15
cluster_concentration = 4


In [18]:
def get_3d_density_profile(r3d, mdelta, cdelta, Delta=200, profile_type='nfw', cosmo=None):
    if profile_type=='nfw':
        return ct.rho_nfw_at_r(r3d, mdelta, cdelta, self.cosmo.Omegam, delta=Delta)
    else:
        pass              


In [31]:
def calculate_surface_density(r_proj, mdelta, cdelta, Delta=200, profile_type='nfw', 
                                        cosmo=None):
    """Sigma"""

    Om_m = cosmo['Omega_c']+cosmo['Omega_b']
    
    if profile_type=='nfw':
        return ct.deltasigma.Sigma_nfw_at_R(r_proj, mdelta, cdelta, Om_m, delta=Delta)
    else:
        #return ct.Sigma_at_R(r_proj, mdelta, cdelta, cosmo.Omegam, delta=Delta)
        pass



In [33]:
def calculate_excess_surface_density(r_proj, mdelta, cdelta, Delta=200, profile_type='nfw', cosmo=None):
    """DeltaSigma"""

    Om_m = cosmo['Omega_c']+cosmo['Omega_b']

    
    Sigma = ct.Sigma_at_R(r_proj, mdelta, cdelta, Om_m, delta=Delta)
    return ct.deltasigma.DeltaSigma_at_R(r_proj, r_proj, Sigma, mdelta, cdelta, Om_m, delta=Delta)
 


In [34]:
def _comoving_angular_distance_aexp1_aexp2(cosmo, aexp1, aexp2):
    z1 = 1./aexp1 - 1.
    z2 = 1./aexp2 - 1. 
    from astropy.cosmology import FlatLambdaCDM
    cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
    return cosmo.angular_diameter_distance_z1z2(z1, z2).value
    
    

def get_critical_surface_density(cosmo, z_cluster, z_source):
    c = ccl.physical_constants.CLIGHT
    G = ccl.physical_constants.GNEWT
    aexp_cluster = 1./(1.+z_cluster)
    aexp_src = 1./(1.+z_source)
    d_l = ccl.comoving_angular_distance(cosmo, aexp_cluster)
    d_s = ccl.comoving_angular_distance(cosmo, aexp_src)
    d_ls = _comoving_angular_distance_aexp1_aexp2(cosmo, aexp_cluster, aexp_src)

    # will need to deal with units: distances in Mpc and some CCL constants in SI
    return d_s/(d_l*d_ls) * c*c/(4*np.pi*G)
    

In [35]:
def compute_tangential_shear_profile(r_proj, mdelta, cdelta, z_cluster, z_source, Delta=200, 
                                     profile_type='nfw', cosmo=None,
                                    z_src_model='single plane'):
    '''gamma_t'''
    delta_sigma = calculate_excess_surface_density(r_proj, mdelta, cdelta, Delta=Delta, 
                                                      profile_type=profile_type, cosmo=cosmo)
        
    if z_src_model == 'single plane':
        sigma_c = get_critical_surface_density(cosmo, z_cluster, z_source)
        return delta_sigma / sigma_c
    elif z_src_model == 'known_z_src': # Discrete case
        NotImplementedError('Need to implemnt Beta_s functionality, or average delta_sigma/sigma_c gamma_t = Beta_s*gamma_inf')
    elif z_src_model == 'z_src_distribution' : # Continuous ( from a distribution) case
        NotImplementedError('Need to implement Beta_s calculation from integrating distribution of redshifts in each radial bin')

In [37]:
def compute_convergence_profile(r_proj, mdelta, cdelta, z_cluster, z_source, Delta=200, 
                                     profile_type='nfw', cosmo=None,
                                    z_src_model='single plane'):
    '''kappa'''

    sigma = calculate_surface_density(r_proj, mdelta, cdelta, Delta=Delta, 
                                                      profile_type=profile_type, cosmo=cosmo)
    
    if z_src_model == 'single plane':
        sigma_c = get_critical_surface_density(cosmo, z_cluster, z_source)
        return sigma / sigma_c
    elif z_src_model == 'known_z_src': # Discrete case
        NotImplementedError('Need to implemnt Beta_s functionality, or average sigma/sigma_c kappa_t = Beta_s*kappa_inf')
    elif z_src_model == 'z_src_distribution' : # Continuous ( from a distribution) case
        NotImplementedError('Need to implement Beta_s calculation from integrating distribution of redshifts in each radial bin')

In [38]:
def compute_reduced_tangential_shear_profile(r_proj, mdelta, cdelta, z_cluster, z_source, Delta=200, 
                                     profile_type='nfw', cosmo=None,
                                    z_src_model='single plane'):
    '''g_t'''

    if z_src_model == 'single plane':
        kappa = compute_convergence_profile(r_proj, mdelta, cdelta, z_cluster, z_source, Delta, 
                                         profile_type, cosmo,
                                        z_src_model)
        gamma_t = compute_tangential_shear_profile(r_proj, mdelta, cdelta, z_cluster, z_source, Delta, 
                                         profile_type, cosmo,
                                        z_src_model)
        return gamma_t / (1 - kappa)
    elif z_src_model == 'known_z_src': # Discrete case
        NotImplementedError('Need to implemnt Beta_s functionality, or average sigma/sigma_c kappa_t = Beta_s*kappa_inf')
    elif z_src_model == 'z_src_distribution' : # Continuous ( from a distribution) case
        NotImplementedError('Need to implement Beta_s and Beta_s2 calculation from integrating distribution of redshifts in each radial bin')
    